# Asynchronous Programming in Python

One of the hardest general problems in modern software development is programming concurrent streams of execution. If you simply want to execute steps A-Z in order, that's straightforward; but if you want to parallelize steps that are not contingent on each other, that gets hard.

## Concurrent Execution

The following example demonstrates the point:

In [1]:
import random
import time

# call random.seed(0) to ensure every re-run of this
# program has the same sleep times for each job
random.seed(0)

def function_that_takes_time(name):
    # note that this could be waiting for an API request or
    # web request or any other event that this computer simply
    # waits for the completion of
    start = time.time()
    print(f'starting: {name} ({start})')
    time.sleep(random.uniform(1, 2))
    end = time.time()
    print(f'completed: {name} ({end} -> {end - start} seconds)')

def do_synchronous():
    start = time.time()
    print(f'starting: ({start})')
    function_that_takes_time('A')
    function_that_takes_time('B')
    function_that_takes_time('C')
    end = time.time()
    print(f'completed ({end} -> {end - start} seconds)')

do_synchronous()

starting: (1567364332.185318)
starting: A (1567364332.1853929)
completed: A (1567364334.030619 -> 1.8452260494232178 seconds)
starting: B (1567364334.030831)
completed: B (1567364335.7930722 -> 1.7622411251068115 seconds)
starting: C (1567364335.7932649)
completed: C (1567364337.2177181 -> 1.4244532585144043 seconds)
completed (1567364337.217862 -> 5.032543897628784 seconds)


Note a few things:

* all steps are completed in the order they were started
* each step completes before the next one starts
* the total time is basically the sum of the job times
* your computer is sitting idle for most of that total time

**Question:** what if we could call our jobs in a non-blocking way?

**non-blocking job:** a job that allows other jobs to have the CPU any time it is going to take a long time to complete


## Historical Approaches

This basic desire is the heart of parallel programming of all kinds. Some you have probably heard of:

* multi-threading
* multi-process/distributed computing
* GPU parallelization

These each use different mechanisms to achieve the same end goal: reducing the time it takes to complete time-intensive jobs. All the above approaches split the program into pieces that can compute side-by-side and provide mechanisms to start and stop jobs, get the results of jobs, and otherwise manage and monitor flows of execution.

Does that sounds confusing? That's because it is. Learning to think in parallel flows of execution is hard.


## A Special Case

But there are a few types of problems that come up again and again and that can greatly benefit from parallelizing jobs. The most common example of this are problems that are I/O-bound: waiting for a request to arrive, waiting for a response to arrive, waiting for a timer to expire, waiting for the user to do something.

Do you see what these all have in common? They are waiting on some other thing to happen, and are therefore bound by input and output... they are I/O bound.


## `asyncio`

The `asyncio` package is built into Python's later versions, starting with 3.4. `asyncio` is built on the notion of *coroutines*. Think of coroutines as functions that are given a baton (control) until they decide to hand the baton off because they are temporarily stuck.

**coroutine:** a method that takes control as long as it has instructions that can be executed immediately and then gives up control until the next time it is given control

`asyncio` uses two important keywords to facilitate creating and calling coroutines: `async` and `await`.

Mark a function as `async` if it should be treated as a coroutine:

```python
async def function_that_takes_time(name):
    # ... implementation
    return
```

Call an async function with `await` to "pass the baton" to a coroutine and to be given control back only when the coroutine has completed:

```python
result = await function_that_takes_time('A')
```


## A Practical Example

Rewriting the program at the top of this article to use coroutines looks like this:

In [2]:
import asyncio
import random
import time

# call random.seed(0) to ensure every re-run of this
# program has the same sleep times for each job
random.seed(0)

async def async_function_that_takes_time(name):
    # note that this could be waiting for an API request or
    # web request or any other event that must complete before
    # this function can make progress
    start = time.time()
    print(f'starting: {name} ({start})')
    await asyncio.sleep(random.uniform(1, 2))
    end = time.time()
    print(f'completed: {name} ({end} -> {end - start} seconds)')

async def do_asynchronous():
    start = time.time()
    print(f'starting: ({start})')
    futures = (
        async_function_that_takes_time('A'),
        async_function_that_takes_time('B'),
        async_function_that_takes_time('C'),
    )
    print('jobs created...')
    await asyncio.gather(*futures)
    end = time.time()
    print(f'completed ({end} -> {end - start} seconds)')

loop = asyncio.get_event_loop()
await loop.create_task(do_asynchronous())

starting: (1567364337.231812)
jobs created...
starting: A (1567364337.231969)
starting: B (1567364337.232019)
starting: C (1567364337.232068)
completed: C (1567364338.6573539 -> 1.425285816192627 seconds)
completed: B (1567364338.994304 -> 1.7622849941253662 seconds)
completed: A (1567364339.07688 -> 1.8449108600616455 seconds)
completed (1567364339.077247 -> 1.8454349040985107 seconds)


If you look at the output, you'll notice a few things:

* the jobs don't start until I tell them to
    * even though I call them before printing "jobs created", none of them actually do anything until I `await` them
* the jobs complete in a total time that is barely longer than the time of the longest job
    * each job will wait for between 1 and 2 seconds, so the total cumulative time waiting will be somwhere between 3 and 6 seconds
    * since the waiting is happening in parallel, the actual clock time is much lower than this total
* the jobs complete in an order that may not match the starting order
    * jobs complete when they complete, regardless of what order they were called in
* every call to a job that is async is preceeded with the `await` keyword
    * this includes calls to external libraries that are async-able, like `asyncio.sleep()`


So what is `asyncio` actually doing?


## Asynchronous vs Threading

A single CPU core can only actually do one thing at a time.

Threads, the most common parallelizing strategy, involves starting multiple separate paths of execution that are only partially aware of each other (using thread control primitives like locks and shared context) and that each get their turns to run for periods of time. A single thread never gives up control; instead, the OS determines which thread (from all threads across all programs the computer is running) is running on which core at any time.

The asyncronous paradigm is different: only a single thread is ever running, and the thread is always executing the things that are not blocked, and then sitting in an idle state whenever every single running job is blocked until one of them unblocks. Since all code is in a single thread, and since the currently-running job will continue to have control until it `await`s some other call, the program is much simpler to think about and understand.


## Event Loop

The overall control structure that handles the async program is called the *event loop*. The event loop holds all coroutines and manages coroutine interactions with the underlying OS, coordinating system calls that block and returning control to the coroutine when those system calls unblock. Coroutines that are unblocked are in one queue, coroutines that are blocked are in a different queue, and whenever a coroutine `await`s another coroutine, that coroutine is added to the queue and the calling coroutine is moved from the unblocked queue to the blocked queue. The coroutine that is now at the front of the unblocked queue is given control back until it `await`s something else, and then the whole process repeats.

You start the event loop like this:

```python
loop = asyncio.get_event_loop()
await loop.run_until_complete(some_async_call())
```

**Note:** our programs in this document are running in a context (Jupyter) that already has an event loop, so instead of **starting** the event loop, we are just adding a coroutine to the existing loop

**Note:** there are actually many ways to start the event loop, each with different purposes, as described in [Running and stopping the loop](https://docs.python.org/3/library/asyncio-eventloop.html#running-and-stopping-the-loop)

Once the loop is started, you can add coroutines to the loop from anywhere:

```python
loop = asyncio.get_event_loop()
await loop.create_task(some_async_call())
```


## Futures

A *future* can be thought of as the result of a coroutine call that will return a value in the future. When we call `async_function_that_takes_time()`, we are not getting the result of the coroutine, we are getting a future that will eventually contain the result of the coroutine. The coroutine doesn't actually get a chance to run until it gets to the head of the unblocked queue. This is why we can make all three coroutine calls but none of them start until we await them.


## Gathering Coroutines

One thing you probably noticed is that we call `asyncio.gather()` with a list of futures. The `gather()` method is a single coroutine that you can `await` that will not return until all coroutines passed to it have completed (successfully or unsuccessfully). This allows the current coroutine to kick off several parallel jobs and then simply wait for completion.


## Tasks

A *task* is a coroutine you add to the event loop directly (instead of awaiting it), and whose exit represents the end of your job. The event loop will continue to run the task (alongside all other tasks) until it exits cleanly or throws an exception. Once a task has been created, you can interrogate the task object to see its status (running, exited, error, etc) and additionally manipulate it.


## Blocking

The instant you `await` a coroutine, the calling coroutine blocks (halts and is placed in the blocked queue), and the called coroutine is added to the running queue but not started. When the running queue has run all other tasks in the running queue until each of them has also blocked or completed, then this called coroutine is given the chance to run until it blocks.

If your coroutine *never* calls an `await`able method, it will continue to be in control and all other coroutines will be blocked until it exits. Don't do this.

The following program is identical to the previous one except that it calls the `time.sleep()` method instead of `await`ing a call to `asyncio.sleep()`; see how this serializes the output again, as no job gets to actually start until the job before it has run to completion (by never `await`ing a blocking call):

In [3]:
import asyncio
import random
import time

# call random.seed(0) to ensure every re-run of this
# program has the same sleep times for each job
random.seed(0)

async def async_function_that_takes_time(name):
    # note that this could be waiting for an API request or
    # web request or any other event that must complete before
    # this function can make progress
    start = time.time()
    print(f'starting: {name} ({start})')
    time.sleep(random.uniform(1, 2))
    end = time.time()
    print(f'completed: {name} ({end} -> {end - start} seconds)')

async def do_asynchronous():
    start = time.time()
    print(f'starting: ({start})')
    futures = (
        async_function_that_takes_time('A'),
        async_function_that_takes_time('B'),
        async_function_that_takes_time('C'),
    )
    print('jobs created...')
    await asyncio.gather(*futures)
    end = time.time()
    print(f'completed ({end} -> {end - start} seconds)')

loop = asyncio.get_event_loop()
await loop.create_task(do_asynchronous())

starting: (1567364339.089548)
jobs created...
starting: A (1567364339.089802)
completed: A (1567364340.934981 -> 1.8451790809631348 seconds)
starting: B (1567364340.935393)
completed: B (1567364342.695232 -> 1.7598388195037842 seconds)
starting: C (1567364342.6955059)
completed: C (1567364344.119828 -> 1.4243221282958984 seconds)
completed (1567364344.1201842 -> 5.0306360721588135 seconds)


## Exceptions

One of the more confusing aspects of async programming is handling exceptions. Remember that each coroutine is running independently on the asyncio queues, but you may have called several coroutines together (using `gather()`) or simply added independent tasks on the event loop (using `create_task()`).

Every future could potentially end with an uncaught exception. It is up to the calling coroutine to handle this; [Detect never-retrieved exceptions](https://docs.python.org/3/library/asyncio-dev.html#detect-never-retrieved-exceptions) describes this in more detail, and gives the following example:

In [4]:
import asyncio

async def bug():
    raise Exception('not consumed')

async def main():
    asyncio.create_task(bug())

asyncio.create_task(main())

<Task pending coro=<main() running at <ipython-input-4-5bcbd6db21ce>:6>>

Task exception was never retrieved
future: <Task finished coro=<bug() done, defined at <ipython-input-4-5bcbd6db21ce>:3> exception=Exception('not consumed')>
Traceback (most recent call last):
  File "<ipython-input-4-5bcbd6db21ce>", line 4, in bug
    raise Exception('not consumed')
Exception: not consumed


**Note:** the `main()` coroutine is creating a separate task on the event loop and never looking at the result; this is why the exception is ultimately uncaught anywhere

Since debugging unhandled exceptions in async code is so difficult, `asyncio` has mechanisms for enabling debugging and seeing much richer information about what went wrong. How to do this is beyond the scope of this tutorial.


## Summary

The `asyncio` built-in package makes it much easier than before to write *extremely* fast parallel code, especially when that code is I/O bound. Any time your program could do several things at once and those things are all predominantly blocked by latencies outside of your program (network calls, disk calls, etc), you should look to leverage this package and the async paradigm.

## References

* [`asyncio` — Asynchronous I/O](https://docs.python.org/3/library/asyncio.html)
* [Developing with `asyncio`](https://docs.python.org/3/library/asyncio-dev.html)
* [Event Loop](https://docs.python.org/3/library/asyncio-eventloop.html)
* [Coroutines and Tasks](https://docs.python.org/3/library/asyncio-task.html)
